In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import Counter

import lightgbm

import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline


# Загрузка данных

In [5]:
#DATA_DIR = 'data'   # Home
#DATA_DIR = '../input/data-science-bowl-2019' # Kaggle

PROCESSED_DATA_DIR = 'processed/557'  # Work
# PROCESSED_DATA_DIR = '../input/data-processing-consolidated'  # Kaggle

RANDOM_SEED = 17

In [6]:
import pickle
import os

featureset = None
with open(os.path.join(PROCESSED_DATA_DIR, 'train_processed.pkl'), 'rb') as fi:
    train = pickle.load(fi)

train.shape

(17687, 550)

In [7]:
from sklearn.model_selection import cross_val_score, GroupShuffleSplit
from sklearn.metrics import cohen_kappa_score

# Определение требуемой метрики 'quadratic weighted kappa'
scorer = lambda estimator, x, y : cohen_kappa_score(estimator.predict(x), y, weights='quadratic')

## Регрессия LightGBM с выбором порогов

In [8]:
from numba import jit

@jit
def qwk(a1, a2):
    """
    Source: https://www.kaggle.com/c/data-science-bowl-2019/discussion/114133#latest-660168

    :param a1:
    :param a2:
    :param max_rat:
    :return:
    """
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e


In [9]:
from functools import partial
import scipy as sp

class OptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']
    
    
class MultistartOptimizedRounder(object):
    """
    An optimizer for rounding thresholds
    to maximize Quadratic Weighted Kappa (QWK) score
    # https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved
    """
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        """
        Get loss according to
        using current coefficients
        
        :param coef: A list of coefficients that will be used for rounding
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

        return -qwk(y, X_p)

    def fit(self, X, y):
        """
        Optimize rounding thresholds
        
        :param X: The raw predictions
        :param y: The ground truth labels
        """
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coefs = [[0.5, 1.5, 2.5],
                         [1.1, 1.6, 2.2],
                         [1.2, 1.7, 2.1]]
        best_val = 10.0                    # значение берется с обратным знаком
        best_coef = None
        for coef in initial_coefs:
            cur_coef = sp.optimize.minimize(loss_partial, coef, method='nelder-mead')
            cur_val = self._kappa_loss(cur_coef['x'], X, y)
            print('Val:', cur_val)
            if cur_val < best_val:         # значение берется с обратным знаком
                print('New best!')
                best_val = cur_val
                best_coef = cur_coef
        self.coef_ = best_coef

    def predict(self, X, coef):
        """
        Make predictions with specified thresholds
        
        :param X: The raw predictions
        :param coef: A list of coefficients that will be used for rounding
        """
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])


    def coefficients(self):
        """
        Return the optimized coefficients
        """
        return self.coef_['x']

In [10]:
from sklearn.base import BaseEstimator, MetaEstimatorMixin, ClassifierMixin, clone

class ClassifierWrapper(BaseEstimator, MetaEstimatorMixin, ClassifierMixin):
    
    def __init__(self, base_estimator, estimator_params):
        super().__init__()
        self.base_estimator = base_estimator
        self.estimator_params = estimator_params        
        self.optimized_rounder = MultistartOptimizedRounder()

    def make_estimator(self):
        estimator = clone(self.base_estimator)
        #estimator.set_params(**dict((p, getattr(self, p))
        #                            for p in self.estimator_params))
        estimator.set_params(**self.estimator_params)
        return estimator        
        
    def fit(self, X, y):
        self.model = self.make_estimator().fit(X, y)
        y_pred = self.model.predict(X)
        self.optimized_rounder.fit(y_pred.reshape(-1,), y)
        # coefficients = optR.coefficients()
        print('Found bounds:', self.optimized_rounder.coefficients())
        
    def predict(self, X):
        y_pred = self.model.predict(X)
        return self.optimized_rounder.predict(y_pred.reshape(-1, ), self.optimized_rounder.coefficients())
        

In [11]:
X = train.drop(columns=['installation_id', 'timestamp', 
                        'accuracy_group', 'accuracy', 'num_correct', 'num_incorrect'])
y = train[['accuracy_group']]


X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]


In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GroupKFold

splitter = GroupKFold(5)

# Определение требуемой метрики 'quadratic weighted kappa'
scorer = lambda estimator, x, y : cohen_kappa_score(estimator.predict(x), y, weights='quadratic')

# Baseline: модель с исходными параметрами

In [13]:

# from Andrew Lukyanenko
parameters = {'n_estimators':2000,
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.04,
            'feature_fraction': 0.9,
         'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'random_state': RANDOM_SEED,
            'early_stopping_rounds': 100,
            'n_jobs': 16
            }

scores = []
for train_idxs, test_idxs in splitter.split(X, y, groups=train.installation_id):
    train_X = X.iloc[train_idxs, :]
    train_y = y.iloc[train_idxs, :]
    test_X = X.iloc[test_idxs, :]
    test_y = y.iloc[test_idxs, :]
    train_data = lightgbm.Dataset(train_X, label=train_y, categorical_feature=['session_title', 'world'])
    test_data = lightgbm.Dataset(test_X, label=test_y)
    model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=test_data)
    rounder = MultistartOptimizedRounder()
    rounder.fit(model.predict(train_X).reshape(-1), train_y)
    y_pred = rounder.predict(model.predict(test_X).reshape(-1, ), rounder.coefficients())
    scores.append(cohen_kappa_score(y_pred, test_y, weights='quadratic'))

scores = np.array(scores)    
print(scores)    
print(scores.mean())    
print(scores.std())    
    


/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['session_title', 'world']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 1.23915
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1.2244
[3]	valid_0's rmse: 1.21042
[4]	valid_0's rmse: 1.19817
[5]	valid_0's rmse: 1.18599
[6]	valid_0's rmse: 1.17823
[7]	valid_0's rmse: 1.16646
[8]	valid_0's rmse: 1.15892
[9]	valid_0's rmse: 1.14856
[10]	valid_0's rmse: 1.1393
[11]	valid_0's rmse: 1.13047
[12]	valid_0's rmse: 1.12442
[13]	valid_0's rmse: 1.11593
[14]	valid_0's rmse: 1.10839
[15]	valid_0's rmse: 1.10159
[16]	valid_0's rmse: 1.09551
[17]	valid_0's rmse: 1.08983
[18]	valid_0's rmse: 1.0847
[19]	valid_0's rmse: 1.07976
[20]	valid_0's rmse: 1.07454
[21]	valid_0's rmse: 1.07062
[22]	valid_0's rmse: 1.06621
[23]	valid_0's rmse: 1.0622
[24]	valid_0's rmse: 1.05851
[25]	valid_0's rmse: 1.05479
[26]	valid_0's rmse: 1.05166
[27]	valid_0's rmse: 1.04903
[28]	valid_0's rmse: 1.04608
[29]	valid_0's rmse: 1.04334
[30]	valid_0's rmse: 1.04111
[31]	valid_0's rmse: 1.03876
[32]	valid_0's rmse: 1.0366
[33]	valid_0's rmse: 1.0

<ipython-input-8-7e6567382b89>:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "qwk" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-8-7e6567382b89> (13)

File "<ipython-input-8-7e6567382b89>", line 13:
def qwk(a1, a2):
    <source elided>
    """
    max_rat = 3
    ^

  @jit
<ipython-input-8-7e6567382b89>:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "qwk" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-8-7e6567382b89>", line 21:
def qwk(a1, a2):
    <source elided>
    o = 0
    for k in range(a1.shape[0]):
    ^

  @jit
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "qwk" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-8-7

Val: [-0.70809897]
New best!
Val: [-0.73510759]
New best!
Val: [-0.73475886]


/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['session_title', 'world']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's rmse: 1.24673
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1.23138
[3]	valid_0's rmse: 1.21698
[4]	valid_0's rmse: 1.20364
[5]	valid_0's rmse: 1.19143
[6]	valid_0's rmse: 1.18381
[7]	valid_0's rmse: 1.17192
[8]	valid_0's rmse: 1.16509
[9]	valid_0's rmse: 1.15482
[10]	valid_0's rmse: 1.14585
[11]	valid_0's rmse: 1.13712
[12]	valid_0's rmse: 1.13237
[13]	valid_0's rmse: 1.1248
[14]	valid_0's rmse: 1.11716
[15]	valid_0's rmse: 1.11
[16]	valid_0's rmse: 1.10333
[17]	valid_0's rmse: 1.09681
[18]	valid_0's rmse: 1.09163
[19]	valid_0's rmse: 1.08622
[20]	valid_0's rmse: 1.08132
[21]	valid_0's rmse: 1.07688
[22]	valid_0's rmse: 1.07262
[23]	valid_0's rmse: 1.06866
[24]	valid_0's rmse: 1.06493
[25]	valid_0's rmse: 1.06135
[26]	valid_0's rmse: 1.05808
[27]	valid_0's rmse: 1.05475
[28]	valid_0's rmse: 1.05136
[29]	valid_0's rmse: 1.04871
[30]	valid_0's rmse: 1.04584
[31]	valid_0's rmse: 1.04347
[32]	valid_0's rmse: 1.04101
[33]	valid_0's rmse: 1.

[71]	valid_0's rmse: 0.988803
[72]	valid_0's rmse: 0.98855
[73]	valid_0's rmse: 0.988223
[74]	valid_0's rmse: 0.987912
[75]	valid_0's rmse: 0.987635
[76]	valid_0's rmse: 0.987398
[77]	valid_0's rmse: 0.987353
[78]	valid_0's rmse: 0.987319
[79]	valid_0's rmse: 0.986847
[80]	valid_0's rmse: 0.986378
[81]	valid_0's rmse: 0.986134
[82]	valid_0's rmse: 0.985877
[83]	valid_0's rmse: 0.985675
[84]	valid_0's rmse: 0.985529
[85]	valid_0's rmse: 0.98537
[86]	valid_0's rmse: 0.985309
[87]	valid_0's rmse: 0.985259
[88]	valid_0's rmse: 0.985127
[89]	valid_0's rmse: 0.984795
[90]	valid_0's rmse: 0.984611
[91]	valid_0's rmse: 0.984564
[92]	valid_0's rmse: 0.984356
[93]	valid_0's rmse: 0.98415
[94]	valid_0's rmse: 0.984047
[95]	valid_0's rmse: 0.984
[96]	valid_0's rmse: 0.983911
[97]	valid_0's rmse: 0.983775
[98]	valid_0's rmse: 0.983549
[99]	valid_0's rmse: 0.983505
[100]	valid_0's rmse: 0.983628
[101]	valid_0's rmse: 0.983809
[102]	valid_0's rmse: 0.983701
[103]	valid_0's rmse: 0.98365
[104]	valid_0

[104]	valid_0's rmse: 0.96366
[105]	valid_0's rmse: 0.963824
[106]	valid_0's rmse: 0.963579
[107]	valid_0's rmse: 0.963345
[108]	valid_0's rmse: 0.963291
[109]	valid_0's rmse: 0.963451
[110]	valid_0's rmse: 0.963507
[111]	valid_0's rmse: 0.963104
[112]	valid_0's rmse: 0.963141
[113]	valid_0's rmse: 0.963037
[114]	valid_0's rmse: 0.962833
[115]	valid_0's rmse: 0.962652
[116]	valid_0's rmse: 0.962263
[117]	valid_0's rmse: 0.962231
[118]	valid_0's rmse: 0.962095
[119]	valid_0's rmse: 0.961955
[120]	valid_0's rmse: 0.961965
[121]	valid_0's rmse: 0.961729
[122]	valid_0's rmse: 0.961806
[123]	valid_0's rmse: 0.961813
[124]	valid_0's rmse: 0.9615
[125]	valid_0's rmse: 0.961572
[126]	valid_0's rmse: 0.961464
[127]	valid_0's rmse: 0.961345
[128]	valid_0's rmse: 0.961325
[129]	valid_0's rmse: 0.961194
[130]	valid_0's rmse: 0.96103
[131]	valid_0's rmse: 0.961105
[132]	valid_0's rmse: 0.961098
[133]	valid_0's rmse: 0.961164
[134]	valid_0's rmse: 0.961057
[135]	valid_0's rmse: 0.961184
[136]	valid_

[128]	valid_0's rmse: 0.993979
[129]	valid_0's rmse: 0.994186
[130]	valid_0's rmse: 0.99419
[131]	valid_0's rmse: 0.99429
[132]	valid_0's rmse: 0.994377
[133]	valid_0's rmse: 0.994381
[134]	valid_0's rmse: 0.994095
[135]	valid_0's rmse: 0.994185
[136]	valid_0's rmse: 0.994306
[137]	valid_0's rmse: 0.994284
[138]	valid_0's rmse: 0.994262
[139]	valid_0's rmse: 0.994157
[140]	valid_0's rmse: 0.994497
[141]	valid_0's rmse: 0.994541
[142]	valid_0's rmse: 0.994587
[143]	valid_0's rmse: 0.994627
[144]	valid_0's rmse: 0.994384
[145]	valid_0's rmse: 0.994421
[146]	valid_0's rmse: 0.994298
[147]	valid_0's rmse: 0.994249
[148]	valid_0's rmse: 0.994218
[149]	valid_0's rmse: 0.994168
[150]	valid_0's rmse: 0.994229
[151]	valid_0's rmse: 0.994243
[152]	valid_0's rmse: 0.994232
[153]	valid_0's rmse: 0.994228
[154]	valid_0's rmse: 0.99407
[155]	valid_0's rmse: 0.993939
[156]	valid_0's rmse: 0.993935
[157]	valid_0's rmse: 0.994191
[158]	valid_0's rmse: 0.994154
[159]	valid_0's rmse: 0.994243
[160]	valid

# Настройка параметров

In [14]:
import optuna

In [15]:
def evaluate_parameters(params):
    scores = []
    for seed_delta in range(2):
        params_copy = params.copy()
        params_copy['random_state'] = RANDOM_SEED + seed_delta
        for train_idxs, test_idxs in splitter.split(X, y, groups=train.installation_id):
            train_X = X.iloc[train_idxs, :]
            train_y = y.iloc[train_idxs, :]
            test_X = X.iloc[test_idxs, :]
            test_y = y.iloc[test_idxs, :]
            train_data = lightgbm.Dataset(train_X, label=train_y, categorical_feature=['session_title', 'world'])
            test_data = lightgbm.Dataset(test_X, label=test_y)
            model = lightgbm.train(params_copy,
                                   train_data,
                                   verbose_eval=False)
            rounder = MultistartOptimizedRounder()
            rounder.fit(model.predict(train_X).reshape(-1), train_y)
            y_pred = rounder.predict(model.predict(test_X).reshape(-1, ), rounder.coefficients())
            scores.append(cohen_kappa_score(y_pred, test_y, weights='quadratic'))

    scores = np.array(scores)
    print(scores)    
    print(scores.mean())
    print(scores.std())
    return scores[1:-1].mean()

def objective(trial):
    param = {
        'n_estimators': 500,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'verbose': 0,
        'random_state': RANDOM_SEED,
        #'early_stopping_rounds': 100,
        'n_jobs': 16,
        
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-2, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-2, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),     # subsample   
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 3),                     # subsample_freq
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 0.32),
    }

    # TODO pruning callbacks 
    return evaluate_parameters(param)
    

In [ ]:
study = optuna.create_study(study_name='557_LGBM_Tuning_Paranoidal', direction='maximize', storage='sqlite:///557_lgbm_tuning.db')
study.optimize(objective, n_trials = 200)

[I 2020-01-22 09:55:18,146] A new study created with name: 557_LGBM_Tuning_Paranoidal
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning:

Using categorical_feature in Dataset.



Val: [-0.83778371]
New best!
Val: [-0.86017278]
New best!
Val: [-0.85992764]
Val: [-0.85646493]
New best!
Val: [-0.85662562]
New best!
Val: [-0.85685384]
New best!
Val: [-0.86057587]
New best!
Val: [-0.86075745]
New best!
Val: [-0.86056369]
Val: [-0.85151799]
New best!
Val: [-0.85154544]
New best!
Val: [-0.85166549]
New best!
Val: [-0.86000641]
New best!
Val: [-0.85980333]
Val: [-0.8598495]


/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning:

Using categorical_feature in Dataset.



Val: [-0.85711828]
New best!
Val: [-0.86162901]
New best!
Val: [-0.86128578]
Val: [-0.85508579]
New best!
Val: [-0.85302258]
Val: [-0.85463425]
Val: [-0.86256551]
New best!
Val: [-0.862772]
New best!
Val: [-0.86296298]
New best!
Val: [-0.83190663]
New best!
Val: [-0.85627013]
New best!
Val: [-0.85512057]
Val: [-0.83836501]
New best!
Val: [-0.85940149]
New best!
Val: [-0.85914419]
[0.58399101 0.56812965 0.58678069 0.61693987 0.57295431 0.5833687
 0.56442322 0.58452956 0.62102144 0.57978121]
0.5861919676922778
0.017863117980843106


[I 2020-01-22 10:00:44,216] Finished trial#0 resulted in value: 0.5872684311242395. Current best value is 0.5872684311242395 with parameters: {'bagging_fraction': 0.7508523485904761, 'bagging_freq': 3, 'feature_fraction': 0.5488593130138897, 'lambda_l1': 0.10858314506668078, 'lambda_l2': 0.7911901767523005, 'learning_rate': 0.043623482938099194, 'max_depth': 9, 'min_child_samples': 93, 'num_leaves': 63}.
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning:

Using categorical_feature in Dataset.



...

Val: [-0.70411791]
New best!
Val: [-0.71976581]
New best!
Val: [-0.71953659]
Val: [-0.72188063]
New best!
Val: [-0.72197677]
New best!
Val: [-0.72205402]
New best!
Val: [-0.69633003]
New best!
Val: [-0.72113773]
New best!
Val: [-0.72113931]
New best!
Val: [-0.71260019]
New best!
Val: [-0.71306495]
New best!
Val: [-0.71260687]
Val: [-0.72304118]
New best!
Val: [-0.7223865]
Val: [-0.72280782]


/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning:

Using categorical_feature in Dataset.



Val: [-0.71928562]
New best!
Val: [-0.71963268]
New best!
Val: [-0.71850405]
Val: [-0.72114977]
New best!
Val: [-0.72103928]
Val: [-0.7209692]
Val: [-0.72064151]
New best!
Val: [-0.72178823]
New best!
Val: [-0.72159083]
Val: [-0.71369748]
New best!
Val: [-0.71357802]
Val: [-0.71399191]
New best!
Val: [-0.6969195]
New best!
Val: [-0.7216909]
New best!
Val: [-0.72150712]
[0.59936622 0.58461265 0.59975945 0.63975402 0.58443006 0.59220496
 0.5848188  0.59386903 0.6356704  0.58573183]
0.600021742055618
0.019659115889665887


[I 2020-01-22 23:13:19,000] Finished trial#121 resulted in value: 0.601889920751966. Current best value is 0.6039176826227299 with parameters: {'bagging_fraction': 0.6102822288673977, 'bagging_freq': 1, 'feature_fraction': 0.6175300262006813, 'lambda_l1': 2.536388722221168, 'lambda_l2': 1.7018708200539778, 'learning_rate': 0.01072548886976055, 'max_depth': 10, 'min_child_samples': 75, 'num_leaves': 35}.
/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning:

Found `n_estimators` in params. Will use it instead of argument

/home/paniquex/anaconda3/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning:

Using categorical_feature in Dataset.



Val: [-0.72921417]
New best!
Val: [-0.72936222]
New best!
Val: [-0.72721461]
Val: [-0.72835541]
New best!
Val: [-0.72785034]
Val: [-0.72784557]
